In [1]:
#!pip module -d path


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Importing libraries
    </div>

In [2]:
import os
os.chdir('/kaggle/input/segmentation-models-pytorch-whl-file/whl_files/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4')
!python setup.py install
import pretrainedmodels

/opt/conda/lib/python3.10/site-packages/setuptools/dist.py:744: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please av

/kaggle/input/segmentation-models-pytorch-whl-file/whl_files/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:255: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if block_type is 'proj':
/kaggle/input/segmentation-models-pytorch-whl-file/whl_files/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif block_type is 'down':
/kaggle/input/segmentation-models-pytorch-whl-file/whl_files/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:262: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert block_type is 'normal'


In [3]:
os.chdir('/kaggle/input/segmentation-models-pytorch-whl-file/whl_files/efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1')
!python setup.py install
import efficientnet_pytorch

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/py

In [4]:
!pip install --no-index --find-links /kaggle/input/segmentation-models-pytorch-whl-file/whl_files/ segmentation-models-pytorch --no-deps


Looking in links: /kaggle/input/segmentation-models-pytorch-whl-file/whl_files/
Processing /kaggle/input/segmentation-models-pytorch-whl-file/whl_files/segmentation_models_pytorch-0.3.3-py3-none-any.whl


In [5]:
import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp
from tqdm import tqdm
import time
import glob

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Setting seed for reproducibility
    </div>

In [6]:
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Setting device
    </div>

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Defining vessel dataset class
    </div>

In [8]:
class VesselDataset(Dataset):
    def __init__(self, data_folder, mode='train',transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.mode = mode
        self.images_folder = os.path.join(data_folder, 'images')
        if self.mode == "train":
            self.labels_folder = os.path.join(data_folder, 'labels')
        self.image_files = os.listdir(self.images_folder)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_folder, self.image_files[idx])
        if self.mode == "train":
            label_name = os.path.join(self.labels_folder, self.image_files[idx])
        
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if self.mode == "train":
            label = Image.open(label_name).convert('L')

        if self.transform:
            image = self.transform(image)
            if self.mode == "train":
                label = self.transform(label)
        if self.mode == "train":
            return image, label
        else:
            return image


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Defining transforms and train-val datasets
    </div>

In [9]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),
])

train_dataset = VesselDataset('/kaggle/input/blood-vessel-segmentation/train/kidney_1_dense', transform=transform)
train_dataset += VesselDataset('/kaggle/input/blood-vessel-segmentation/train/kidney_1_voi', transform=transform)
train_dataset += VesselDataset('/kaggle/input/blood-vessel-segmentation/train/kidney_3_sparse', transform=transform)
val_dataset = VesselDataset('/kaggle/input/blood-vessel-segmentation/train/kidney_2', transform=transform)


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Moving cached resnet encoder to root path
    </div>

In [10]:
cache_path = '/root/.cache/torch/hub/checkpoints/'
source_path = '/kaggle/input/unet-model/resnet34-333f7ec4.pth'
if not os.path.exists(cache_path):
    os.makedirs(cache_path)
shutil.copy(source_path, cache_path)


'/root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth'

<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Defining Unet model and creating dataloaders for train and val split
    </div>

In [11]:
model = smp.Unet(
    encoder_name="resnet34",  
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
)
checkpoint_path = '/kaggle/input/unet-model/unet_checkpoint.pth'

model.load_state_dict(torch.load(checkpoint_path))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Moving the model to device and training for 5 epochs
    </div>

In [12]:
model.to(device)

num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for step, (images, labels) in tqdm(enumerate(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        if step%1000==0:
            print("Step-{},Loss-{}".format(step,loss.item()))
            
        optimizer.step()
        break

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation - Epoch {epoch + 1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, labels)
            break
    val_loss /= len(val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}")

#torch.save(model.state_dict(), checkpoint_path)

Epoch 1/1: 0it [00:10, ?it/s]


Step-0,Loss-0.5145243406295776


Validation - Epoch 1/1:   0%|          | 0/35 [00:03<?, ?it/s]

Epoch 1/1, Loss: 0.5145, Validation Loss: 0.0167


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 RLE encode and decode helper functions
    </div>

In [13]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


<div style="background-color:#FFF199; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌 Generating submission by predicting on test set
    </div>

In [14]:
submission_df = pd.read_csv('/kaggle/input/blood-vessel-segmentation/sample_submission.csv')
test_data_folders = glob.glob('/kaggle/input/blood-vessel-segmentation/test/*')

test_dataset = VesselDataset(test_data_folders[0], transform=transform,mode="test")

for folder in test_data_folders[1:]:
    test_dataset += VesselDataset(folder, transform=transform,mode="test")

test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

model.eval()
with torch.no_grad():
    for i, (images) in tqdm(enumerate(test_loader), desc="Generating Submission"):
        images = images.to(device)
        outputs = model(images)
        predictions = torch.sigmoid(outputs)

        for j in range(predictions.shape[0]):
            rle_mask = rle_encode(predictions[j].cpu().numpy() > 0.5)
            submission_df.at[i * 4 + j, 'rle'] = rle_mask

submission_df.to_csv('/kaggle/working/submission.csv', index=False)


Generating Submission: 2it [00:00,  7.04it/s]


In [15]:
submission_df

,id,rle
0,kidney_5_0000,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
1,kidney_5_0001,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
2,kidney_6_0000,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
3,kidney_6_0001,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
4,NaN,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
5,NaN,5 248 254 3 262 1 504 4 511 1 1277 2 1286 2 12...
